In [2]:
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math
import subprocess
import shutil
import os
import sys

DIRECTORY_PATH = os.getcwd()
MATRIX_DIRECTORY_PATH = f'{DIRECTORY_PATH}/matrices'

# AIS Danish maritime dataset
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "processed_data"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "traj"


class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": DATABASE_NAME,
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = TPOINT_TABLE_NAME
            self.id_column_name = TPOINT_ID_COLUMN_NAME
            self.tpoint_column_name = TPOINT_COLUMN_NAME                  
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str})
                        AND a.{self.tpoint_column_name} IS NOT NULL;
                    """
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            self.log(e)


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()




FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 1 # To not overload the memory, we only take a percentage of the ships in the database
TIME_DELTA_SIZE = 240 # Number of frames associated to one Time delta
GRANULARITY = timedelta(minutes=1) # Time delta between two frames
SRID = 4326
FPS = 60



  

In [3]:
db = Database_connector()

start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY )

timestamps = [start_date + i * GRANULARITY for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [4]:
timestamps

[datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 1),
 datetime.datetime(2023, 6, 1, 0, 2),
 datetime.datetime(2023, 6, 1, 0, 3),
 datetime.datetime(2023, 6, 1, 0, 4),
 datetime.datetime(2023, 6, 1, 0, 5),
 datetime.datetime(2023, 6, 1, 0, 6),
 datetime.datetime(2023, 6, 1, 0, 7),
 datetime.datetime(2023, 6, 1, 0, 8),
 datetime.datetime(2023, 6, 1, 0, 9),
 datetime.datetime(2023, 6, 1, 0, 10),
 datetime.datetime(2023, 6, 1, 0, 11),
 datetime.datetime(2023, 6, 1, 0, 12),
 datetime.datetime(2023, 6, 1, 0, 13),
 datetime.datetime(2023, 6, 1, 0, 14),
 datetime.datetime(2023, 6, 1, 0, 15),
 datetime.datetime(2023, 6, 1, 0, 16),
 datetime.datetime(2023, 6, 1, 0, 17),
 datetime.datetime(2023, 6, 1, 0, 18),
 datetime.datetime(2023, 6, 1, 0, 19),
 datetime.datetime(2023, 6, 1, 0, 20),
 datetime.datetime(2023, 6, 1, 0, 21),
 datetime.datetime(2023, 6, 1, 0, 22),
 datetime.datetime(2023, 6, 1, 0, 23),
 datetime.datetime(2023, 6, 1, 0, 24),
 datetime.datetime(2023, 6, 1, 0, 2

In [5]:
x_min = -180
y_min = -90
x_max = 180
y_max = 90

start_frame= 240
end_frame = 479

arguments = [start_frame, end_frame, PERCENTAGE_OF_OBJECTS, x_min, y_min, x_max, y_max]
arguments = [str(arg) for arg in arguments]
arguments += [timestamps_strings[0]  , str(len(timestamps)), "MINUTE", MATRIX_DIRECTORY_PATH, DATABASE_NAME, TPOINT_TABLE_NAME, TPOINT_ID_COLUMN_NAME, TPOINT_COLUMN_NAME]
            

In [6]:
arguments

['240',
 '479',
 '1',
 '-180',
 '-90',
 '180',
 '90',
 '2023-06-01 00:00:00',
 '1439',
 'MINUTE',
 '/home/ali/QGIS-MobilityDB/experiment9_sql_optimization/matrices',
 'mobilitydb',
 'processed_data',
 'MMSI',
 'traj']

In [6]:
# Command to execute Program B
command = ['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file_processed.py', *arguments]
result = subprocess.run(command, capture_output=True, text=True)

In [7]:
result

CompletedProcess(args=['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file_processed.py', '240', '479', '1', '-180', '-90', '180', '90', '2023-06-01 00:00:00', '1439', 'MINUTE', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrices', 'mobilitydb', 'processed_data', 'MMSI', 'traj'], returncode=0, stdout='================================================================     Matrix 240 created in 23.64022731781006 seconds, 709.2068195343018 frames for 30 fps animation.\n', stderr='')

import numpy as np

# Load the .npy file with allow_pickle set to True
loaded_matrix = np.load('matrix_0.npy', allow_pickle=True)

print(loaded_matrix)


# Debugging process B code

In [7]:
sysargs =arguments 
sysargs

['240',
 '479',
 '1',
 '-180',
 '-90',
 '180',
 '90',
 '2023-06-01 00:00:00',
 '1439',
 'MINUTE',
 '/home/ali/QGIS-MobilityDB/experiment9_sql_optimization/matrices',
 'mobilitydb',
 'processed_data',
 'MMSI',
 'traj']

In [8]:

import numpy as np
from shapely.geometry import Point
from pymeos.db.psycopg import MobilityDB

from pymeos import *
import os
import sys
from datetime import timedelta, datetime
from pymeos import *
import time

logs = ""
now = time.time()

FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


args = sysargs
logs += f"Args: {args}\n"
begin_frame = int(args[0])
end_frame = int(args[1])
TIME_DELTA_SIZE = end_frame - begin_frame + 1
PERCENTAGE_OF_OBJECTS = float(args[2])


SRID = 4326


DATABASE_NAME = args[11]
TPOINT_TABLE_NAME = args[12]
TPOINT_ID_COLUMN_NAME = args[13]
TPOINT_COLUMN_NAME = args[14]



class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": DATABASE_NAME,
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = TPOINT_TABLE_NAME
            self.id_column_name = TPOINT_ID_COLUMN_NAME
            self.tpoint_column_name = TPOINT_COLUMN_NAME               
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str});
                    """
            self.cursor.execute(query)
            # print(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            # print(e)
            pass


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()


MATRIX_DIRECTORY_PATH = "/home/ali/matrices"
file_name = f"/home/ali/matrices/matrix_{begin_frame}.npy"


  
Time_granularities = {
                    # "MILLISECOND" : timedelta(milliseconds=1),
                      "SECOND" : timedelta(seconds=1),
                      "MINUTE" : timedelta(minutes=1),
                    #   "HOUR" : timedelta(hours=1),
                    }


# check if file does't already exist

pymeos_initialize()
db = Database_connector()

x_min = float(args[3])
y_min = float(args[4])
x_max = float(args[5])
y_max = float(args[6])

start_date = args[7]
start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')


total_frames = int(args[8])
GRANULARITY = Time_granularities[args[9]]

timestamps = []
for i in range(total_frames): 
    timestamps.append(start_date + i*GRANULARITY)



p_start = timestamps[begin_frame]
p_end = timestamps[end_frame]
# print(p_start, p_end, x_min, y_min, x_max, y_max)
now_db = time.time()
rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)    

print(f"Time to fetch tpoints : {time.time() - now_db} s")    
        

Time to fetch tpoints : 2.7915709018707275 s


In [13]:
%%timeit
i=3
rows[i][0].values()

4.9 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
p_start

datetime.datetime(2023, 6, 1, 4, 0)

In [18]:
from pymeos import *

pymeos_initialize()



In [31]:
x1 = TGeomPointInst(point=(0, 0), timestamp=timestamps[0])
x2 = TGeomPointInst(point=(1, 1), timestamp=timestamps[TIME_DELTA_SIZE-1])
x2

TGeomPointInst(POINT(1 1)@2023-06-01 03:59:00+02)

In [32]:
traj = TGeomPointSeq.from_instants([x1, x2], upper_inc=True)

In [33]:
traj_resampled = traj.temporal_sample(start=timestamps[0],duration= GRANULARITY)
traj_resampled

TGeomPointSeq({POINT(0 0)@2023-06-01 00:00:00+02, POINT(0.00418410041841 0.00418410041841)@2023-06-01 00:01:00+02, POINT(0.00836820083682 0.00836820083682)@2023-06-01 00:02:00+02, POINT(0.01255230125523 0.01255230125523)@2023-06-01 00:03:00+02, POINT(0.01673640167364 0.01673640167364)@2023-06-01 00:04:00+02, POINT(0.02092050209205 0.02092050209205)@2023-06-01 00:05:00+02, POINT(0.02510460251046 0.02510460251046)@2023-06-01 00:06:00+02, POINT(0.02928870292887 0.02928870292887)@2023-06-01 00:07:00+02, POINT(0.03347280334728 0.03347280334728)@2023-06-01 00:08:00+02, POINT(0.03765690376569 0.03765690376569)@2023-06-01 00:09:00+02, POINT(0.0418410041841 0.0418410041841)@2023-06-01 00:10:00+02, POINT(0.04602510460251 0.04602510460251)@2023-06-01 00:11:00+02, POINT(0.05020920502092 0.05020920502092)@2023-06-01 00:12:00+02, POINT(0.054393305439331 0.054393305439331)@2023-06-01 00:13:00+02, POINT(0.058577405857741 0.058577405857741)@2023-06-01 00:14:00+02, POINT(0.062761506276151 0.062761506276

In [34]:
empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)


In [39]:

    trajectory_array = np.array([point.wkt for point in traj_resampled.values()])

In [40]:
trajectory_array

array(['POINT (0 0)', 'POINT (0.00418410041841 0.00418410041841)',
       'POINT (0.00836820083682 0.00836820083682)',
       'POINT (0.01255230125523 0.01255230125523)',
       'POINT (0.01673640167364 0.01673640167364)',
       'POINT (0.02092050209205 0.02092050209205)',
       'POINT (0.02510460251046 0.02510460251046)',
       'POINT (0.02928870292887 0.02928870292887)',
       'POINT (0.03347280334728 0.03347280334728)',
       'POINT (0.03765690376569 0.03765690376569)',
       'POINT (0.0418410041841 0.0418410041841)',
       'POINT (0.04602510460251 0.04602510460251)',
       'POINT (0.05020920502092 0.05020920502092)',
       'POINT (0.054393305439331 0.054393305439331)',
       'POINT (0.058577405857741 0.058577405857741)',
       'POINT (0.062761506276151 0.062761506276151)',
       'POINT (0.066945606694561 0.066945606694561)',
       'POINT (0.071129707112971 0.071129707112971)',
       'POINT (0.075313807531381 0.075313807531381)',
       'POINT (0.079497907949791 0.0794

In [42]:
traj_resampled.start_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0)

datetime.datetime(2023, 6, 1, 0, 0)

In [43]:

time_ranges = timestamps
now = time.time()

for i in range(len(rows)):
    start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) 
    end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) 

    trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
    matrix[i, start_index:end_index+1] = trajectory_array

print(f"Time to fill the matrix : {time.time() - now} s")

Time to fill the matrix : 29.986719131469727 s


In [ ]:


TGeogPointSeq.from_instants(x, upper_inc=True)

218 
589



In [13]:
empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

time_ranges = timestamps
now = time.time()

for i in range(len(rows)):
    if rows[i][0] is not None:
        try:
            traj = rows[i][0]
            num_instants = traj.num_instants()
            if num_instants == 1:
                single_timestamp = traj.timestamps()[0].replace(tzinfo=None).replace(second=0, microsecond=0)
                index = time_ranges.index(single_timestamp) - begin_frame
                matrix[i][index] = traj.values()[0].wkt
            
            elif num_instants >= 2:
                traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY)
                
                start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame
                end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame
        
                trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
                matrix[i, start_index:end_index+1] = trajectory_array
    
        except:
            continue


db.close()
pymeos_finalize()
total_time = time.time() - now
frames_for_30_fps= 30 * total_time
print(f"Matrix {begin_frame} created in {total_time} seconds, {frames_for_30_fps} frames for 30 fps animation.")
# logs += f"time to create and fill the matrix {begin_frame}: {total_time} seconds\n"


================================================================     Matrix 240 created in 26.292837619781494 seconds, 788.7851285934448 frames for 30 fps animation.


# With resample in the database query

In [7]:

import numpy as np
from shapely.geometry import Point
from pymeos.db.psycopg import MobilityDB

from pymeos import *
import os
import sys
from datetime import timedelta, datetime
from pymeos import *
import time

logs = ""
now = time.time()

FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


args = sysargs
logs += f"Args: {args}\n"
begin_frame = int(args[0])
end_frame = int(args[1])
TIME_DELTA_SIZE = end_frame - begin_frame + 1
PERCENTAGE_OF_OBJECTS = float(args[2])


SRID = 4326


DATABASE_NAME = args[11]
TPOINT_TABLE_NAME = args[12]
TPOINT_ID_COLUMN_NAME = args[13]
TPOINT_COLUMN_NAME = args[14]



class Database_connector2:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": DATABASE_NAME,
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = TPOINT_TABLE_NAME
            self.id_column_name = TPOINT_ID_COLUMN_NAME
            self.tpoint_column_name = TPOINT_COLUMN_NAME               
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    WITH trajectories as (
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        ) as trajectory
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str}))

                    SELECT tsample(trajectory, INTERVAL '1 minute', TIMESTAMP '2023-06-01 00:00:00')  AS resampled_trajectory
                        FROM 
                            trajectories ;
 
                    """
            self.cursor.execute(query)
            # print(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            # print(e)
            pass


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()


MATRIX_DIRECTORY_PATH = "/home/ali/matrices"
file_name = f"/home/ali/matrices/matrix_{begin_frame}.npy"


  
Time_granularities = {
                    # "MILLISECOND" : timedelta(milliseconds=1),
                      "SECOND" : timedelta(seconds=1),
                      "MINUTE" : timedelta(minutes=1),
                    #   "HOUR" : timedelta(hours=1),
                    }


# check if file does't already exist

pymeos_initialize()
db = Database_connector2()

x_min = float(args[3])
y_min = float(args[4])
x_max = float(args[5])
y_max = float(args[6])

start_date = args[7]
start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')


total_frames = int(args[8])
GRANULARITY = Time_granularities[args[9]]

timestamps = []
for i in range(total_frames): 
    timestamps.append(start_date + i*GRANULARITY)



p_start = timestamps[begin_frame]
p_end = timestamps[end_frame]
# print(p_start, p_end, x_min, y_min, x_max, y_max)
now_db = time.time()
rows = db.get_subset_of_tpoints(p_start, p_end, x_min, y_min, x_max, y_max)    

print(f"Time to fetch tpoints : {time.time() - now_db} s")    
        

Time to fetch tpoints : 2.6934139728546143 s


In [8]:
%load_ext line_profiler

In [17]:
def opr(rows):
    empty_point_wkt = Point()  # "POINT EMPTY"
    matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

    time_ranges = timestamps
    now = time.time()

    for i in range(len(rows)):
        if rows[i][0] is not None:
            try:
                traj_resampled = rows[i][0]
                # num_instants = traj_resampled.num_instants()
                # if num_instants == 1:
                #     # print(f"{i} has one instant")
                #     single_timestamp = traj_resampled.timestamps()[0].replace(tzinfo=None).replace(second=0, microsecond=0)
                #     index = time_ranges.index(single_timestamp) - begin_frame
                #     matrix[i][index] = traj.values()[0].wkt
                
                # elif num_instants >= 2:
                    # traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY)
                    
                start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame
                end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame
                vals = traj_resampled.values()
                trajectory_array = np.array([point.wkt for point in vals])
                matrix[i, start_index:end_index+1] = trajectory_array
        
            except:
                continue


    # db.close()
    # pymeos_finalize()
    total_time = time.time() - now
    frames_for_30_fps= 30 * total_time
    print(f"Matrix {begin_frame} created in {total_time} seconds, {frames_for_30_fps} frames for 30 fps animation.")
    # logs += f"time to create and fill the matrix {begin_frame}: {total_time} seconds\n"

    

In [18]:
%lprun -f opr opr(rows)

Matrix 240 created in 42.29663419723511 seconds, 1268.8990259170532 frames for 30 fps animation.


Timer unit: 1e-09 s

Total time: 42.2668 s
File: /tmp/ipykernel_17693/2884772419.py
Function: opr at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def opr(rows):
     2         1      73167.0  73167.0      0.0      empty_point_wkt = Point()  # "POINT EMPTY"
     3         1   11737600.0    1e+07      0.0      matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)
     4                                           
     5         1       1298.0   1298.0      0.0      time_ranges = timestamps
     6         1       1935.0   1935.0      0.0      now = time.time()
     7                                           
     8      5822    4032885.0    692.7      0.0      for i in range(len(rows)):
     9      5821    5873313.0   1009.0      0.0          if rows[i][0] is not None:
    10      3998    1022013.0    255.6      0.0              try:
    11      3998    1232917.0    308.4      0.0           

In [19]:
empty_point_wkt = Point().wkt  # "POINT EMPTY"
matrix = np.full((len(rows), TIME_DELTA_SIZE), empty_point_wkt, dtype=object)

time_ranges = timestamps
now = time.time()

for i in range(len(rows)):
    if rows[i][0] is not None:
        try:
            traj_resampled = rows[i][0]
            # num_instants = traj_resampled.num_instants()
            # if num_instants == 1:
            #     # print(f"{i} has one instant")
            #     single_timestamp = traj_resampled.timestamps()[0].replace(tzinfo=None).replace(second=0, microsecond=0)
            #     index = time_ranges.index(single_timestamp) - begin_frame
            #     matrix[i][index] = traj.values()[0].wkt
            
            # elif num_instants >= 2:
                # traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= GRANULARITY)
                
            start_index = time_ranges.index( traj_resampled.start_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame
            end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None).replace(second=0, microsecond=0) ) - begin_frame

            trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
            matrix[i, start_index:end_index+1] = trajectory_array
    
        except:
            # print(i)
            continue


# db.close()
# pymeos_finalize()
total_time = time.time() - now
frames_for_30_fps= 30 * total_time
print(f"Matrix {begin_frame} created in {total_time} seconds, {frames_for_30_fps} frames for 30 fps animation.")
# logs += f"time to create and fill the matrix {begin_frame}: {total_time} seconds\n"


Matrix 240 created in 24.599054098129272 seconds, 737.9716229438782 frames for 30 fps animation.


In [10]:
pip install line_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.4/748.4 kB 4.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
rows[409]

(TGeomPointSeq({POINT(10.987850259451445 54.373360044477394)@2023-06-01 07:59:00+02}),)

In [9]:
single_timestamp = traj.timestamps()[0].replace(tzinfo=None).replace(second=0, microsecond=0)
index = time_ranges.index(single_timestamp) - begin_frame
index

NameError: name 'traj' is not defined

In [19]:
len(rows)

5821

In [9]:
np.count_nonzero(matrix != 'POINT EMPTY')

865790

# Measuring nditer vs for loop for feature generation

In [29]:
%%timeit

empty_point_wkt = Point().wkt  # "POINT EMPTY"
# create a numpy array of size len(ids_list) with empty_point_wkt
starting_points = np.full((1, 5821), empty_point_wkt, dtype=object)

qgis_fields_list = []

for wkt in np.nditer(starting_points, flags=['refs_ok']):
    feat = ["vlayer_fields"]
    feat.append("datetime_obj")  # Set its attributes
    # Create geometry from WKT string
    feat.append(wkt.item())
    qgis_fields_list.append(feat)
        

962 µs ± 39.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%%timeit

empty_point_wkt = Point().wkt  # "POINT EMPTY"

qgis_fields_list = []

for i in range(5821):
    feat = ["vlayer_fields"]
    feat.append("datetime_obj")  # Set its attributes
    # Create geometry from WKT string
    feat.append(empty_point_wkt)
    qgis_fields_list.append(feat)

378 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Removing old files

In [62]:
import shutil

# Path to the directory
directory_path = f'/home/ali/matrices/'

shutil.rmtree(directory_path)


In [63]:
# create directory to store the matrices
os.makedirs(directory_path)